# Import Bibliotheken

In [71]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

### Package Versionen

In [72]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [73]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [74]:
comment_df = pd.read_excel('../../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [75]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [76]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [77]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [78]:
# next(iter(train_set))

## Dataloader

### Tokenisierung

In [79]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

In [80]:
# Beispiel für Anwendung des tokenizer
# sentence = "This isn't a very long example."
# tokenizer(sentence)

### Erstellung Vokabular

In [81]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [82]:
# Länge des Trainungsets
len(train_set)

2798

In [83]:
tokens = yield_tokens(train_set)

In [84]:
next(iter(tokens))

['wir',
 'waren',
 'auf',
 'der',
 'durchreise',
 'vom',
 'urlaub',
 'nach',
 'hause',
 'und',
 'wurden',
 'von',
 'frau',
 'ludwig',
 'sehr',
 'freundlich',
 'und',
 'zuvorkommend',
 'bedient',
 'da',
 'das',
 'in',
 'der',
 'heutigen',
 'zeit',
 'leider',
 'nicht',
 'mehr',
 'selbstverständlich',
 'ist',
 'möchten',
 'wir',
 'uns',
 'auf',
 'diesem',
 'wege',
 'noch',
 'einmal',
 'sehr',
 'herzlich',
 'bei',
 'ihr',
 'bedanken',
 'marko',
 'seuß']

In [85]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [86]:
len(vocab)

2470

In [87]:
vocab(['Hallo', 'sehr', 'Personal'])

[0, 3, 0]

In [88]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [89]:
onehot = torch.zeros(1,len(vocab))

In [90]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[29, 0, 47]

In [91]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [92]:
import torch
import torch.nn

In [93]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [94]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [95]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [96]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [97]:
unflat = [[x] for x in flat_set]

In [98]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1, specials=["<unk>"])

label_vocab.set_default_index(vocab["<unk>"])

In [99]:
label_vocab.lookup_token(3)

'DigitalFueling'

#### Dictionary erstellen

In [100]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Shop': 0,
 'Erscheinungsbild': 1,
 'AdBlue': 2,
 'Kartenakzeptanzen': 3,
 'Paketservice': 4,
 'Luft': 5,
 'Staubsauger': 6,
 'Pricing': 7,
 'Öffnungszeiten': 8,
 'Personal': 9,
 'WLAN': 10,
 'E-Fuels': 11,
 'Nacht-/Tankautomat': 12,
 'Kraftstoffauswahl': 13,
 'Tankpool': 14,
 'Waschanlage': 15,
 'Werkstatt': 16,
 'SB-Waschboxen': 17,
 'E-Mobilität': 18,
 'Zapfsäulen': 19,
 'Bistro': 20,
 'DigitalFueling': 21,
 'Sanitär': 22,
 'Verkehrsanbindung': 23}

#### Tensor

In [101]:
doc2 = label_list_export[2]
doc2

['Personal', 'Waschanlage']

In [102]:
pos_vals = [label_dict[val] for val in doc2]
pos_vals

[9, 15]

In [103]:
labels = torch.LongTensor(pos_vals)
labels

tensor([ 9, 15])

In [104]:
y_onehot = nn.functional.one_hot(labels, num_classes=24)
y_onehot

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [105]:
y_onehot = y_onehot.sum(dim=0).float()
y_onehot

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [106]:
# labels = torch.tensor([1,5,8,0])
# labels = labels.unsqueeze(0)
# target = torch.zeros(labels.size(0), 24).scatter_(1, labels, 1.)
# print(target)

### Erstellung Dataloader

In [107]:
# label_pipeline = lambda x: int(x) -1

In [108]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        # label_list.append(label_pipeline(_label))
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [109]:
train_loader = DataLoader(
 train_set, batch_size=16,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [110]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [111]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [112]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.MSELoss()

## Initialisierung Optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate, momentum=0.9)

## Definition der Epochen
num_epochs = 250

In [113]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

loss_hist = {}
accuracy_hist = {}

In [114]:
for (text, label) in train_loader:
    #print(label[[0][0]])
    #print(text)
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,


In [115]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().numpy()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.0
Training Accuracy for epoch 1:0.0
Training Accuracy for epoch 2:0.0
Training Accuracy for epoch 3:0.0032165832817554474
Training Accuracy for epoch 4:0.014653324149549007
Training Accuracy for epoch 5:0.04431736841797829
Training Accuracy for epoch 6:0.14045746624469757
Training Accuracy for epoch 7:0.18263044953346252
Training Accuracy for epoch 8:0.2194424569606781
Training Accuracy for epoch 9:0.23409578204154968
Training Accuracy for epoch 10:0.2548248767852783
Training Accuracy for epoch 11:0.26340243220329285
Training Accuracy for epoch 12:0.27340957522392273
Training Accuracy for epoch 13:0.2787705361843109
Training Accuracy for epoch 14:0.2823445200920105
Training Accuracy for epoch 15:0.28484630584716797
Training Accuracy for epoch 16:0.28842028975486755
Training Accuracy for epoch 17:0.2905646860599518
Training Accuracy for epoch 18:0.2916368842124939
Training Accuracy for epoch 19:0.2937812805175781
Training Accuracy for epoch 20:0.293781280

In [116]:
total_train_accuracy = train_accuracy.compute()

c:\Users\loedu\anaconda3\envs\ai_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: The ``compute`` method of metric Accuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


### Auswertung Fehlerentwicklung

In [117]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.087111
1,1,0.085956
2,2,0.083519
3,3,0.075028
4,4,0.066489


In [118]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
)
# glue('train-loss-team', train_chart,display=True)
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [119]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,0.003217
4,4,0.014653


In [120]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
)
accuracy_chart

alt.Chart(...)

## Evaluierung

In [121]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2470, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=25, bias=True)
)

### Gesamtevaluierung

In [122]:
test_loader = DataLoader(test_set, batch_size= 16, shuffle=True,collate_fn=collate_batch)

In [123]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [124]:
total_valid_accuracy

tensor(0.3953)

### Evaluierung nach Klassen

In [125]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=25
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [126]:
total_valid_accuracy

tensor([0.6739, 0.9908, 0.8649, 0.9958,    nan, 0.9983, 0.8557, 0.9683, 0.9233,
        0.9808, 0.9716, 0.9950, 0.8015, 0.8557, 0.9883, 0.9691, 0.8616, 0.9933,
        0.9992, 0.9683, 0.9983, 0.8982, 0.9917, 0.9925, 0.9691])

### Evaluierung nach der Anzahl der Epochen

In [127]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=collate_batch)

In [128]:
test_loader = DataLoader(test_set, batch_size=16, shuffle=True, collate_fn=collate_batch)

In [129]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [130]:
learning_rate = 0.05

loss_fn = nn.MSELoss()

optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9)

num_epochs = 250

In [131]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [132]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_train_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = loss_fn(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()

Training Accuracy for epoch 0: 0.0
Validation Accuracy for epoch 0: 0.0
Training Accuracy for epoch 1: 0.0
Validation Accuracy for epoch 1: 0.0
Training Accuracy for epoch 2: 0.0
Validation Accuracy for epoch 2: 0.0
Training Accuracy for epoch 3: 0.007505360990762711
Validation Accuracy for epoch 3: 0.010008340701460838
Training Accuracy for epoch 4: 0.008220157586038113
Validation Accuracy for epoch 4: 0.01834862306714058
Training Accuracy for epoch 5: 0.06933523714542389
Validation Accuracy for epoch 5: 0.09591326117515564
Training Accuracy for epoch 6: 0.1508220136165619
Validation Accuracy for epoch 6: 0.17097581923007965
Training Accuracy for epoch 7: 0.1915654093027115
Validation Accuracy for epoch 7: 0.19349457323551178
Training Accuracy for epoch 8: 0.21801286935806274
Validation Accuracy for epoch 8: 0.20683903992176056
Training Accuracy for epoch 9: 0.23731236159801483
Validation Accuracy for epoch 9: 0.2226855754852295
Training Accuracy for epoch 10: 0.25232309103012085
Vali

In [133]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "Train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "Test"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [134]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
    x = alt.X('Epoch', title='Anzahl Epochen'),
    y = alt.Y('Loss',title='Mittlerer Fehler'),
    color = alt.Color('Type',title='Typ')
)
train_eval_loss_chart

alt.Chart(...)

In [135]:
valid_accuracy_hist

{0: array(0., dtype=float32),
 1: array(0., dtype=float32),
 2: array(0., dtype=float32),
 3: array(0.01000834, dtype=float32),
 4: array(0.01834862, dtype=float32),
 5: array(0.09591326, dtype=float32),
 6: array(0.17097582, dtype=float32),
 7: array(0.19349457, dtype=float32),
 8: array(0.20683904, dtype=float32),
 9: array(0.22268558, dtype=float32),
 10: array(0.23102586, dtype=float32),
 11: array(0.24770643, dtype=float32),
 12: array(0.2585488, dtype=float32),
 13: array(0.25771475, dtype=float32),
 14: array(0.2618849, dtype=float32),
 15: array(0.26522103, dtype=float32),
 16: array(0.26438698, dtype=float32),
 17: array(0.26855713, dtype=float32),
 18: array(0.26688907, dtype=float32),
 19: array(0.2702252, dtype=float32),
 20: array(0.2735613, dtype=float32),
 21: array(0.27189323, dtype=float32),
 22: array(0.27189323, dtype=float32),
 23: array(0.2768974, dtype=float32),
 24: array(0.27439532, dtype=float32),
 25: array(0.28023353, dtype=float32),
 26: array(0.28106755, dt

In [136]:
train_accuracy_hist

{0: array(0., dtype=float32),
 1: array(0., dtype=float32),
 2: array(0., dtype=float32),
 3: array(0.00750536, dtype=float32),
 4: array(0.00822016, dtype=float32),
 5: array(0.06933524, dtype=float32),
 6: array(0.15082201, dtype=float32),
 7: array(0.19156541, dtype=float32),
 8: array(0.21801287, dtype=float32),
 9: array(0.23731236, dtype=float32),
 10: array(0.2523231, dtype=float32),
 11: array(0.26304504, dtype=float32),
 12: array(0.2730522, dtype=float32),
 13: array(0.27841315, dtype=float32),
 14: array(0.28270194, dtype=float32),
 15: array(0.28377414, dtype=float32),
 16: array(0.2873481, dtype=float32),
 17: array(0.2894925, dtype=float32),
 18: array(0.2905647, dtype=float32),
 19: array(0.2912795, dtype=float32),
 20: array(0.29342386, dtype=float32),
 21: array(0.29306647, dtype=float32),
 22: array(0.29556826, dtype=float32),
 23: array(0.29556826, dtype=float32),
 24: array(0.29664046, dtype=float32),
 25: array(0.29771265, dtype=float32),
 26: array(0.29807004, dty

In [137]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [138]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "Train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "Test"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [139]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.000000,Train
1,2,0.000000,Train
2,3,0.007505,Train
3,4,0.008220,Train
4,5,0.069335,Train
...,...,...,...
244,245,0.389491,Test
245,246,0.390325,Test
246,247,0.388657,Test
247,248,0.388657,Test


In [140]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x = alt.X('Epoch',title='Anzahl Epochen'),
    y = alt.Y('Accuracy',title='Genauigkeit'),
    color = alt.Color('Type',title='Typ')
)
train_eval_accuracy_chart

alt.Chart(...)